# 주가 원인분석 훈련 모델 
#기업명 : LG에너지솔루션
#코드 명 : 373220

### 실행 방법 ###
-  뉴스파일 / 댓글 파일 로드
- electra_light 파일 넣어두기
- 뒤에 electra_light 저장경로 변경 후 실행
- 20ep 모델과 10ep 모델 비교

In [ ]:
company_name = 'lgenergy'
company_code = '373220'
company_stopword = ['LG', 'lg', '에너지', '솔루션', '에너지솔루션', '엔솔']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
# 뉴스 파일 로드
df = pd.read_csv('/content/drive/MyDrive/뉴스원인분석_modeling/data/LG에너지솔루션_tagging_0109.csv')
# 댓글 파일 로드
df_bc = pd.read_csv('/content/drive/MyDrive/뉴스원인분석_modeling/data/board_comment/LG에너지솔루션_tagging.csv')
# 폴더생성
os.makedirs(f'/content/drive/MyDrive/뉴스원인분석_modeling/final_news_modeling/{company_name}', exist_ok=True)

# 모델 전처리


In [ ]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
!pip install kss
!pip install scipy==1.8.0
!pip install git+https://github.com/lovit/textrank.git
!pip install konlpy
!pip install 'git+https://github.com/krevas/huggingface_konlpy'

Installing automake (A dependency for mecab-ko)
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1,351 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backp

In [ ]:
# 라이브러리 import
import pandas as pd
from tqdm.notebook import tqdm
from kss import split_sentences
import numpy as np
import os

from konlpy.tag import Mecab
import warnings
from collections import Counter
warnings.filterwarnings("ignore")
mecab = Mecab()

In [ ]:
df

,media,date,date_hour,url,content,photourl,date_cate,날짜,전일비_비율,standard,standard_tagging0.5,standard_tagging0.25
0,연합뉴스,20220127,2022-01-27 17:53,https://n.news.naver.com/mnews/article/001/001...,증시자금 블랙홀 엔솔에 코스피 시총 69조원 증발서울연합뉴스 이미령 기자 공모주 ...,https://imgnews.pstatic.net/image/001/2022/01/...,2022-01-28,2022-01-28,-10.89,-4.209753,0,0
1,연합뉴스,20220127,2022-01-27 18:01,https://n.news.naver.com/mnews/article/001/001...,연합뉴스 이 시각 헤드라인 1800역제안에 수용 양자토론 31일 가닥4자는 다시 ...,https://imgnews.pstatic.net/image/001/2022/01/...,2022-01-28,2022-01-28,-10.89,-4.209753,0,0
2,이데일리,20220127,2022-01-27 18:09,https://n.news.naver.com/mnews/article/018/000...,단독우리사주 대박 엔솔 임직원성과급도 따블로이데일리 성주원 기자 화학051910에서...,https://imgnews.pstatic.net/image/018/2022/01/...,2022-01-28,2022-01-28,-10.89,-4.209753,0,0
3,이데일리,20220127,2022-01-27 19:33,https://n.news.naver.com/mnews/article/018/000...,"에너지솔루션, 조기편입 불발조건 만족 못해이데일리 안혜신 기자 에너지솔루션3732...",NaN,2022-01-28,2022-01-28,-10.89,-4.209753,0,0
4,이데일리,20220127,2022-01-27 20:21,https://n.news.naver.com/mnews/article/018/000...,"엔솔, 따상 불발했지만공모가보다 70 상승이데일리 안혜신 기자 에너지솔루션37322...",https://imgnews.pstatic.net/image/018/2022/01/...,2022-01-28,2022-01-28,-10.89,-4.209753,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5948,헤럴드경제,20221130,2022-11-30 11:32,https://n.news.naver.com/mnews/article/016/000...,두달간 바이코리아...6조 쓸어담았다올해 10월과 11월 두 달간 외국인 투자자가 ...,NaN,2022-11-30,2022-11-30,3.53,1.310257,4,4
5949,서울경제,20221130,2022-11-30 15:41,https://n.news.naver.com/mnews/article/011/000...,데이터로 보는 증시코스피 기관외국인개인 순매수순매도 상위종목11월 30일서울경제 기...,https://imgnews.pstatic.net/image/011/2022/11/...,2022-11-30,2022-11-30,3.53,1.310257,4,4
5950,서울경제,20221130,2022-11-30 13:46,https://n.news.naver.com/mnews/article/011/000...,"특징주대성하이텍, 전기차 주행거리 30 상승 신기술 3세대 코일 현대차 부품 개발중...",NaN,2022-11-30,2022-11-30,3.53,1.310257,4,4
5951,서울경제,20221130,2022-11-30 15:53,https://n.news.naver.com/mnews/article/011/000...,"마감 시황 코스피, 외국인 폭풍 매수에 1.6 상승 마감서울경제 코스피가 외국인 투...",https://imgnews.pstatic.net/image/011/2022/11/...,2022-11-30,2022-11-30,3.53,1.310257,4,4


In [ ]:
# 전처리 함수

def sent_split_list(df) :
  # 문장 단위로 구분
  df_sent = pd.DataFrame()
  date_cate_list = []
  text_rank_content_list = []
  standard_tagging_list = []
  targetpm_list = []
  idx_list = []
  for idx in tqdm(df.index) :
    temp_date = df.loc[idx, 'date_cate']
    temp_sent_total = df.loc[idx, 'content']
    temp_label = df.loc[idx, 'standard_tagging0.25']
    temp_targetpm = df.loc[idx, 'targetpm']
    temp_sent = split_sentences(str(temp_sent_total))
    for sent in temp_sent:
      date_cate_list.append(temp_date)
      text_rank_content_list.append(sent)
      standard_tagging_list.append(temp_label)
      targetpm_list.append(temp_targetpm)
      idx_list.append(idx)
      # temp = pd.DataFrame({'date' : [temp_date],
      #                     'sent' : [sent],
      #                     'target' : [temp_label],
      #                     'targetpm' : [temp_targetpm],
      #                     'news_id' : [idx]
      #                     })
      # df_sent = pd.concat([df_sent, temp])
  # df_sent = df_sent.reset_index(drop=True)
  df_sent['date'] = date_cate_list
  df_sent['sent'] = text_rank_content_list
  df_sent['target'] = standard_tagging_list
  df_sent['targetpm'] = targetpm_list
  df_sent['news_id'] = idx_list
  return df_sent

In [ ]:
df = df[['date_cate', 'content','전일비_비율','standard_tagging0.25']]
df['targetpm'] = np.where(df['전일비_비율'] >= 0, 3, 1)
df['targetpm'] = df['targetpm'].astype('int')
df_sent = sent_split_list(df)
df_sent

  0%|          | 0/5953 [00:00<?, ?it/s]

,date,sent,target,targetpm,news_id
0,2022-01-28,증시자금 블랙홀 엔솔에 코스피 시총 69조원 증발서울연합뉴스 이미령 기자 공모주 ...,0,1,0
1,2022-01-28,27일 한국거래소에 따르면 이날 유가증권시장 시가총액은 총 2천56조8천억원으로 전...,0,1,0
2,2022-01-28,"다만 이는 이날 상장한 에너지솔루션 시가총액118조2천억원이 더해진 규모로, 에너지...",0,1,0
3,2022-01-28,"이날 코스피가 3.50 급락하며 2,610대로 밀려난 것도 이런 이유에서란 분석이다.",0,1,0
4,2022-01-28,서상영 미래에셋증권 연구원은 이날 코스피 하락 요인 중 가장 큰 건 에너지솔루션 수...,0,1,0
...,...,...,...,...,...
125904,2022-11-30,업계에서는 분할 공포증 영향이라는 해석도 나온다.,4,3,5952
125905,2022-11-30,화학의 에너지솔루션 물적 분할 이후 분할악재라는 인식 때문에 주가가 내리는 것이라는...,4,3,5952
125906,2022-11-30,한 투자 업계 고위 임원은 기존 사업이 돈을 벌고 신사업에 투자하다 신사업이 자생력...,4,3,5952
125907,2022-11-30,다만 주가가 지나치게 급락했다는 점에서 외부로 알려지지 않은 내용이 더 있는 것 아...,4,3,5952


In [ ]:
df_sent['target'].value_counts()

2    31128
3    30389
1    26282
0    19764
4    18346
Name: target, dtype: int64

In [ ]:
# df_sent 저장 코드
save_df_sent_path = f'/content/drive/MyDrive/뉴스원인분석_modeling/final_news_modeling/{company_name}/{company_name}_sent_split.csv'
df_sent.to_csv(save_df_sent_path, index=False)

# 모델 훈련

In [ ]:
######## GPU 전환 후 실행 !!! ########

# KoBERT
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
# GPU 토치 버전 설정
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import pandas as pd
import os


In [ ]:
## KoBERT 기초 설정 ##


max_len = 128 
batch_size = 64
warmup_ratio = 0.1
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5


device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 5, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
def train_test_dataframe(df): 
  df = df.sample(frac=1, random_state=2022).reset_index(drop=True)
  df_gr = df_sent.groupby('date')[['target']].mean()
  # 날짜별로 train_test_split
  train_date, test_date = train_test_split(df_gr, test_size=0.1, random_state=2022, stratify=df_gr['target'])
  train_date_list = train_date.index
  test_date_list = test_date.index
  train_data = df_sent[df_sent['date'].isin(train_date_list)][['date','sent', 'target','targetpm','news_id']]
  test_data = df_sent[df_sent['date'].isin(test_date_list)][['date','sent', 'target','targetpm','news_id']]
  return train_data, test_data

def train_sampling(train, train_num=0, sample_labels = 10000) :
  # 훈련데이터 샘플링
  train = train.reset_index(drop=True)
  idx_list = []
  for i in range(0,5):
    train_temp = train[train['target'] == i]
    train_temp_date = train[train['target'] == i]['date'].unique()
    frac_ratio = sample_labels / len(train_temp)
    for date_idx  in train_temp_date :
      train_temp_idx = train_temp[train_temp['date'] == date_idx].sample(frac=frac_ratio, random_state=2022).index
      idx_list += list(train_temp_idx)
  train_temp_batch = train.loc[idx_list, :].sample(frac=1, random_state=2022)
  train_temp_total_list = []
  for i in range(len(train_temp_batch)):
    train_temp_total_list.append([train_temp_batch.iloc[i,1], train_temp_batch.iloc[i,2]])
  data_train_temp = BERTDataset(train_temp_total_list, 0, 1, tok, max_len, True, False)
  train_temp_dataloader = torch.utils.data.DataLoader(data_train_temp, batch_size=batch_size, num_workers=5)
  train_not_use = train.loc[~train.index.isin(idx_list), :]
  # 사용하지 않은 훈련데이터는 추후 예측에서 사용
  return train_temp_dataloader, train_temp_batch, train_not_use

def to_dataloader(df) : 
  df_list = []
  for i in range(len(df)):
    df_list.append([df.iloc[i,1], df.iloc[i,2]])
  df_data = BERTDataset(df_list, 0, 1, tok, max_len, True, False)
  df_loader = torch.utils.data.DataLoader(df_data, batch_size=batch_size, num_workers=5)
  return df_loader

def train_model(train, test, model, save_path, len_train = 0, train_num = 0 , model_path = False) :

  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
      {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
      {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
  optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
  loss_fn = nn.CrossEntropyLoss()

  num_epochs = 40
  t_total = len(train) * 0.8 * num_epochs
  warmup_step = int(t_total * warmup_ratio)
  scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
    

  # 모델 학습 시작
  for e in range(num_epochs):
      train_acc = 0.0

      train_temp_total_list = []
      train_sampled = train.sample(frac=0.8)
      for i in range(len(train_sampled)):
        train_temp_total_list.append([train_sampled.iloc[i,1], train_sampled.iloc[i,2]])
      data_train_temp = BERTDataset(train_temp_total_list, 0, 1, tok, max_len, True, False)
      train_loader = torch.utils.data.DataLoader(data_train_temp, batch_size=batch_size, num_workers=5)

      model.train()
      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_loader)):
          optimizer.zero_grad()
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)
          valid_length= valid_length
          label = label.long().to(device)
          out = model(token_ids, valid_length, segment_ids)
          loss = loss_fn(out, label)
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
          optimizer.step()
          scheduler.step()  # Update learning rate schedule
          train_acc += calc_accuracy(out, label)

          if batch_id % log_interval == 0:
              print("epoch {} batch id {} loss {} train acc {} acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1), train_acc / (batch_id+1)))
      print("epoch {} train acc {} acc {}".format(e+1, train_acc / (batch_id+1), train_acc / (batch_id+1)))
      
      model.eval() # 평가 모드로 변경
      test_acc(model, int(e+1), test, save_path)
  return model


def test_acc(model, epoch, test1, model_path):
  batch_size = 64
  test = test1.copy()
  test = test.reset_index(drop=True)
  test['pred'] = ''

  model.eval()
  unseen_values = test[['sent', 'target']].values
  test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
  test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)
  for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_input)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    out = model(token_ids, valid_length, segment_ids)
    for outn in range(len(out)) :
      pred = int(out[outn].argmax().cpu().numpy())
      idx = batch_size * batch_id + outn
      test.iloc[idx, -1] = pred

  test['pred'] = test['pred'].astype('int')
  # 뉴스별 라벨링
  df_news = test.groupby('news_id')[['target', 'pred', 'targetpm']].mean()
  df_news_idx = df_news.index

  # 뉴스별 최빈값 계산
  for idx in df_news_idx:
    df_news.loc[idx, 'cnt_pred'] = int(test[test['news_id'] == idx ]['pred'].value_counts().idxmax())
    # print(idx, df_news.loc[idx, 'cnt_pred'])
    
  # 뉴스별 값 태깅
  test['news_pred'] = -5
  for idx_nm in range(len(test)) :
    news_idx = test.iloc[idx_nm, 4]
    test.iloc[idx_nm, -1] = df_news.loc[news_idx, 'cnt_pred']

  # 날짜별 최반값 계산
  df_gr = test.groupby('date')[['target', 'news_pred', 'targetpm']].mean()
  df_date = df_gr.index
  for idx in df_date:
    df_gr.loc[idx, 'cnt_pred'] = int(test[test['date'] == idx ]['news_pred'].value_counts().idxmax())

  df_gr['pred_target'] = df_gr['cnt_pred'].apply(lambda x : int(np.round(x)))
  df_gr['pred_pm'] = df_gr['pred_target'].apply(lambda x: 1 if x< 2 else 3)

  df_gr = df_gr.reset_index()
  for idx in df_gr.index :
    if df_gr.loc[idx, 'target'] == df_gr.loc[idx, 'pred_target']:
      temp_eval = True
    else :
      temp_eval = False
    df_gr.loc[idx, 'eval_5'] = temp_eval

  for idx in df_gr.index :
    if df_gr.loc[idx, 'target'] == 0 :
      df_gr.loc[idx, '3target'] = 1
    elif df_gr.loc[idx, 'target'] == 4 :
      df_gr.loc[idx, '3target'] = 3
    else :
      df_gr.loc[idx, '3target'] = int(df_gr.loc[idx, 'target'])

    if df_gr.loc[idx, 'pred_target'] == 0 :
      df_gr.loc[idx, 'pred_3target'] = 1
    elif df_gr.loc[idx, 'pred_target'] == 4 :
      df_gr.loc[idx, 'pred_3target'] = 3
    else :
      df_gr.loc[idx, 'pred_3target'] = int(df_gr.loc[idx, 'pred_target'])

  df_gr['pred_3target'] = df_gr['pred_3target'].apply(lambda x : int(x))

  for idx in df_gr.index :
    if df_gr.loc[idx, '3target'] == df_gr.loc[idx, 'pred_3target']:
      temp_eval = True
    else :
      temp_eval = False
    df_gr.loc[idx, 'eval_3'] = temp_eval

  for idx in df_gr.index :
    if df_gr.loc[idx, 'targetpm'] == df_gr.loc[idx, 'pred_pm']:
      temp_eval = True
    else :
      temp_eval = False
    df_gr.loc[idx, 'eval_pm'] = temp_eval

  acc_5label = float(round(df_gr['eval_5'].sum() / len(df_gr), 2))
  acc_3label = float(round(df_gr['eval_3'].sum() / len(df_gr), 2))
  acc_2label = float(round(df_gr['eval_pm'].sum() / len(df_gr), 2))
  print(f'****************Epoch : {epoch} Test 정확도 ****************', '\n')
  print('5중분류 정확도 :', acc_5label , '\n')
  print('3중분류 정확도 :', acc_3label , '\n')
  print('이진분류 정확도 :', acc_2label , '\n')
  save_txt = ''
  test_valuec = df_gr['pred_target'].value_counts().reset_index().sort_values('index')
  for idx in test_valuec.index :
    temp_label = test_valuec.loc[idx, 'index']
    temp_cnt = test_valuec.loc[idx, 'pred_target']
    temp_txt = f'label : {temp_label} , cnt : {temp_cnt} ' + ' [SEP] '
    save_txt += temp_txt
  os.makedirs(model_path + f'epoch_{epoch}', exist_ok=True)
  with open(model_path + f'{company_name}_model_test_log.txt', 'a', encoding='utf-8') as f :
    txt = f'Epoch : {epoch}, 5중분류 정확도 : {acc_5label}, 3중분류 정확도 : {acc_3label}, 이진분류 정확도 : {acc_2label}'
    txt += (' [SEP] ' + save_txt)
    f.write(txt)
    f.write('\n')
  torch.save(model, model_path + f'epoch_{epoch}' + f'/{company_name}_model_ep{epoch}_new_sampling.pt')
  torch.save(model.state_dict(), model_path + f'epoch_{epoch}' + f'/{company_name}_model_ep{epoch}_new_sampling_state_dict.bin')
  test.to_csv(model_path + f'epoch_{epoch}' + f'/{company_name}_model_ep{epoch}_test.csv', index=False)
  df_gr.to_csv(model_path + f'epoch_{epoch}' + f'/{company_name}_model_ep{epoch}_pred_by_date_test.csv', index=False)


def predict_test(model, test):
  batch_size = 64
  test = test.reset_index(drop=True)
  test['pred'] = ''

  model.eval()
  unseen_values = test[['sent', 'target']].values
  test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
  test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)
  for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_input)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    out = model(token_ids, valid_length, segment_ids)
    for outn in range(len(out)) :
      pred = int(out[outn].argmax().cpu().numpy())
      idx = batch_size * batch_id + outn
      test.iloc[idx, -1] = pred
      if idx % 5000 == 0 :
        print(idx, pred)
  test.head(20)
  return test


def eval_model(test_pred) :
  test_pred['pred'] = test_pred['pred'].astype('int')
  # 뉴스별 라벨링
  df_news = test_pred.groupby('news_id')[['target', 'pred', 'targetpm']].mean()
  df_news_idx = df_news.index

  # 뉴스별 최빈값 계산
  for idx in df_news_idx:
    df_news.loc[idx, 'cnt_pred'] = int(test_pred[test_pred['news_id'] == idx ]['pred'].value_counts().idxmax())
    # print(idx, df_news.loc[idx, 'cnt_pred'])
    
  # 뉴스별 값 태깅
  test_pred['news_pred'] = -5
  for idx_nm in range(len(test_pred)) :
    news_idx = test_pred.iloc[idx_nm, 4]
    test_pred.iloc[idx_nm, -1] = df_news.loc[news_idx, 'cnt_pred']

  # 날짜별 최반값 계산
  df_gr = test_pred.groupby('date')[['target', 'news_pred', 'targetpm']].mean()
  df_date = df_gr.index
  for idx in df_date:
    df_gr.loc[idx, 'cnt_pred'] = int(test_pred[test_pred['date'] == idx ]['news_pred'].value_counts().idxmax())

  df_gr['pred_target'] = df_gr['cnt_pred'].apply(lambda x : int(np.round(x)))
  df_gr['pred_pm'] = df_gr['pred_target'].apply(lambda x: 1 if x< 2 else 3)

  df_gr = df_gr.reset_index()
  for idx in df_gr.index :
    if df_gr.loc[idx, 'target'] == df_gr.loc[idx, 'pred_target']:
      temp_eval = True
    else :
      temp_eval = False
    df_gr.loc[idx, 'eval_5'] = temp_eval

  for idx in df_gr.index :
    if df_gr.loc[idx, 'target'] == 0 :
      df_gr.loc[idx, '3target'] = 1
    elif df_gr.loc[idx, 'target'] == 4 :
      df_gr.loc[idx, '3target'] = 3
    else :
      df_gr.loc[idx, '3target'] = int(df_gr.loc[idx, 'target'])

    if df_gr.loc[idx, 'pred_target'] == 0 :
      df_gr.loc[idx, 'pred_3target'] = 1
    elif df_gr.loc[idx, 'pred_target'] == 4 :
      df_gr.loc[idx, 'pred_3target'] = 3
    else :
      df_gr.loc[idx, 'pred_3target'] = int(df_gr.loc[idx, 'pred_target'])

  df_gr['pred_3target'] = df_gr['pred_3target'].apply(lambda x : int(x))

  for idx in df_gr.index :
    if df_gr.loc[idx, '3target'] == df_gr.loc[idx, 'pred_3target']:
      temp_eval = True
    else :
      temp_eval = False
    df_gr.loc[idx, 'eval_3'] = temp_eval

  for idx in df_gr.index :
    if df_gr.loc[idx, 'targetpm'] == df_gr.loc[idx, 'pred_pm']:
      temp_eval = True
    else :
      temp_eval = False
    df_gr.loc[idx, 'eval_pm'] = temp_eval


  print('5중분류 정확도 :', round(df_gr['eval_5'].sum() / len(df_gr), 2), '\n')
  print('3중분류 정확도 :', round(df_gr['eval_3'].sum() / len(df_gr), 2), '\n')
  print('이진분류 정확도 :', round(df_gr['eval_pm'].sum() / len(df_gr), 2), '\n')

  df_gr.head()
  return test_pred, df_gr

In [ ]:
# 다시 전처리 끝난 데이터 프레임 불러오기
save_df_sent_path = f'/content/drive/MyDrive/뉴스원인분석_modeling/final_news_modeling/{company_name}/{company_name}_sent_split.csv'
df_sent = pd.read_csv(save_df_sent_path)
df_sent['sent'] = df_sent['sent'].astype('str')
print(df_sent['target'].value_counts())
df_sent.head()

2    31128
3    30389
1    26282
0    19764
4    18346
Name: target, dtype: int64


,date,sent,target,targetpm,news_id
0,2022-01-28,증시자금 블랙홀 엔솔에 코스피 시총 69조원 증발서울연합뉴스 이미령 기자 공모주 ...,0,1,0
1,2022-01-28,27일 한국거래소에 따르면 이날 유가증권시장 시가총액은 총 2천56조8천억원으로 전...,0,1,0
2,2022-01-28,"다만 이는 이날 상장한 에너지솔루션 시가총액118조2천억원이 더해진 규모로, 에너지...",0,1,0
3,2022-01-28,"이날 코스피가 3.50 급락하며 2,610대로 밀려난 것도 이런 이유에서란 분석이다.",0,1,0
4,2022-01-28,서상영 미래에셋증권 연구원은 이날 코스피 하락 요인 중 가장 큰 건 에너지솔루션 수...,0,1,0


# 모델학습
- 40 Epoch
- 최소빈도라벨 수 기준 80% 수로 모든 라벨 샘플링
- Epoch 마다 train 데이터에서 80% 샘플링 후 셔플
- test 데이터셋의 예측을 가장 정확히하고 분포가 고른 모델 선정


In [ ]:

save_path = f'/content/drive/MyDrive/뉴스원인분석_modeling/final_news_modeling/{company_name}/{company_name}'
model_path = f'/content/drive/MyDrive/뉴스원인분석_modeling/final_news_modeling/{company_name}/'

# 몇번째 훈련인지 지정
train, test = train_test_dataframe(df_sent)
print('train :', train.shape, 'test :', test.shape)


sample_labels = int(np.floor(df_sent['target'].value_counts().min() * 0.8))


_, train_sampled, train_not_used = train_sampling(train, sample_labels = sample_labels)

model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model.train()

trained_model = train_model(train_sampled, test, model, model_path)




train : (113187, 5) test : (12722, 5)


  0%|          | 1/1415 [00:00<13:20,  1.77it/s]

epoch 1 batch id 1 loss 1.6426713466644287 train acc 0.171875 acc 0.171875


 14%|█▍        | 202/1415 [00:39<03:56,  5.13it/s]

epoch 1 batch id 201 loss 1.6886014938354492 train acc 0.16200248756218905 acc 0.16200248756218905


...

5000 2


159it [00:13, 12.01it/s]

10000 1


199it [00:16, 11.72it/s]


5중분류 정확도 : 0.43 

3중분류 정확도 : 0.62 

이진분류 정확도 : 0.62 



In [ ]:
# 추가 훈련 데이터셋 저장


# 최종 선정 모델 로드
# epoch 27
train, test = train_test_dataframe(df_sent)
trained_model_best = torch.load('/content/drive/MyDrive/뉴스원인분석_modeling/final_news_modeling/lgenergy/epoch_27/lgenergy_model_ep27_new_sampling.pt')

test_pred = predict_test(trained_model_best, test)
test_pred, test_pred_date = eval_model(test_pred)


train_not_used.to_csv(save_path + f'_train_not_used.csv', index=False)
test_pred.to_csv(save_path + f'_test_pred_by_sent.csv', index=False)
test_pred_date.to_csv(save_path + f'_test_pred_by_date.csv', index=False)

# 키워드 추출

In [ ]:
# 중간 로드 코드
# # 키워드 사용 데이터 로드
save_path = f'/content/drive/MyDrive/뉴스원인분석_modeling/final_news_modeling/{company_name}/{company_name}'
_, test = train_test_dataframe(df_sent)
train_not_used = pd.read_csv(save_path + f'_train_not_used.csv')
trained_model_best = torch.load('/content/drive/MyDrive/뉴스원인분석_modeling/final_news_modeling/lgenergy/epoch_27/lgenergy_model_ep27_new_sampling.pt')

In [ ]:
# 20 ep / 30 ep 훈련 데이터셋 다르니 확인!!
# 30ep 모델 사용
print('###########키워드 추출용 사용하지 않은 데이터 예측 ############')
keyword_total = pd.concat([train_not_used, test])
keyword_total['sent'] = keyword_total['sent'].astype('str')
keyword_pred = predict_test(trained_model_best, keyword_total)
keyword_pred, keyword_pred_date = eval_model(keyword_pred)
keyword_pred.to_csv(save_path + f'_test_pred_by_sent.csv', index=False)
keyword_pred_date.to_csv(save_path + f'_test_pred_by_date_total.csv', index=False)

###########키워드 추출용 사용하지 않은 데이터 예측 ############


2it [00:00,  7.34it/s]

0 0


80it [00:10,  8.03it/s]

5000 1


...

45000 3


783it [01:38,  8.16it/s]

50000 3


821it [01:42,  7.98it/s]


5중분류 정확도 : 0.95 

3중분류 정확도 : 0.96 

이진분류 정확도 : 0.87 



In [ ]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
!pip install kss
!pip install scipy==1.8.0
!pip install git+https://github.com/lovit/textrank.git
!pip install konlpy
!pip install 'git+https://github.com/krevas/huggingface_konlpy'

Installing automake (A dependency for mecab-ko)
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu 

In [ ]:
# 라이브러리 import
import pandas as pd
from tqdm.notebook import tqdm
from kss import split_sentences
import numpy as np
from konlpy.tag import Mecab
from konlpy.tag import Okt
okt = Okt()
import warnings
from collections import Counter
warnings.filterwarnings("ignore")
mecab = Mecab()

In [ ]:
# stopword
stop_word_public = ['', ' ',  '기업', '코스피' , '시장', '업종' ,'포인트' ,'연합뉴스','서울경제','서울' ,'경제','헤럴드 경제','더 벨', '이데일리', '연합인포맥스', '매일경제', '매일']

stop_word_company = ['가가', '가까이중국', '가드', '가량', '가바', '가상', '가온', '가운데', '가운데외국인',
       '가운데이익', '가의', '가이드', '가장', '가지', '가지하락', '가치', '각각', '간관', '간담',
       '간밤', '간장', '갈수', '갈수록', '감리', '감마', '감소치료', '강남구', '강릉의료원',
       '강세금융', '강세미국', '개시', '개월', '개인', '개인감소', '개인고가', '개인고용', '개인기관',
       '개인밀러', '개인배당', '개인이모션', '개인이익', '개인전기차', '개인제약', '개인코스닥', '개인판매',
       '개장', '개장출발', '거래', '게임스톱', '겨례', '결론', '결정', '경관', '경기리지', '경우',
       '경인', '경향신문', '계승현', '고객', '고래', '고스', '고작', '고준혁', '골드만삭스', '골든',
       '공간', '공사', '관계자', '관련', '관련대감', '관왕', '광장', '구미영', '구성', '국내',
       '국내외', '권수현', '권효', '권효매도', '권효집값', '규모', '그간', '그나', '그동안', '그랜드',
       '그룹', '그룹그룹', '금요일', '금융투자니콜라', '금융허브', '금은', '금지보합', '금지상향',
       '금지완제', '금지전장', '금지중소형주', '금지출발', '금지현장', '급등외국인', '급락약세', '기가스',
       '기간', '기간산업', '기간전장', '기관', '기관코로나', '기관코스닥', '기대에이치엘비', '기대외국인',
       '기도', '기록', '기반', '기사', '기자', '기존', '기준', '기준상승', '기준피하', '기초',
       '길리어드', '김가영', '김건희', '김경림', '김경림수익률', '김경림피스', '김국배', '김기훈',
       '김대웅', '김동연', '김동원', '김범수', '김병주', '김보', '김상열', '김상조', '김성훈',
       '김소연', '김승', '김아람', '김용재', '김윤지', '김은비', '김은총', '김응태', '김응태종목',
       '김인', '김인매도', '김인상승', '김인종목', '김잔디', '김재', '김재중', '김정유', '김정은',
       '김정주', '김주형', '김지연', '김진호', '김현아', '김형기', '김호준', '나스', '나파', '나흘',
       '나흘외국인', '나흘주간', '남동', '남부', '내년', '내년강세', '내외', '냉장고', '년도', '노반',
       '노재웅', '노총', '노희준', '누가', '누적개발', '뉴스바이오', '다나', '다른', '다만', '다변',
       '다소', '다스', '다시', '다시창업', '다운', '다음', '다음주', '닥터', '단가', '단계',
       '단위', '단장', '달러', '달리', '달새', '닷새', '닷새사외이사', '당국', '당첨', '대감',
       '대감코스닥', '대감혈장', '대감확보', '대량', '대로', '대부분', '대부분상승', '대비', '대전',
       '대체로', '대통령케이드', '대한', '대해', '더니낙폭', '더블유', '도로', '도세', '돌파바이오',
       '동안', '동주', '두번째', '등락의약품', '등락중간', '디바', '디바이', '디아', '디움',
       '디자인상', '디지털', '디테일', '딕주', '딜로', '때문', '또한', '라며', '라운드', '라이',
       '란티스', '량잔량', '량잔량매도', '롄위보', '로나', '로부터', '룩스', '르마트렐', '리서치',
       '리아', '리제', '리지', '리터', '리토', '린지', '마감', '마녀', '마녀보고서', '마는',
       '마마', '마이', '마켓', '만기일물량', '만도', '만이', '만큼', '망치', '매도', '매도수준',
       '매도스케', '매도실적', '매도외국인', '매도의약품', '매도저가', '매도치료', '매도케미칼', '매도코스닥',
       '매도하락', '매물분가', '매수', '매수매수', '매수상승', '매수연초', '매수오후', '매수외국인',
       '매수중국', '매수코스닥', '매수포스코', '매수항체', '매출다애', '머크', '메이', '멜라', '명근',
       '모건', '모두', '모든', '모든박태진', '모리', '모비모습업체', '모습외국인', '모주', '모충',
       '모친상', '무거워', '무단', '문정원', '물량미국', '미국강세', '미국기관', '미국매도', '미국매수',
       '미나', '미노', '미라', '미만', '미만개인', '미만매수', '미만복합', '미스', '미아', '미취',
       '민선', '밀도', '밀러', '밀리', '바로', '바이', '바이오가량', '바짝', '박경훈', '박순',
       '박영철', '박용만', '박원희', '박정수', '박종오', '박태진', '박형수', '반납', '반대', '반도',
       '반도체', '반등', '반락', '반려', '반면', '반복', '반영', '반전', '반토막', '발미달러',
       '발트', '방서', '방인', '방준혁', '배우자', '배지', '백미경', '밸류', '번의', '베바',
       '베터', '벤처캐피탈', '별로', '보삼', '보이', '복씨', '부담급락', '부분', '부사', '부산',
       '부장', '부총', '분기', '분쟁전일', '불댕', '불어', '불확실유가', '붕괴이명철', '비롯', '비르',
       '비메', '비율외국인', '비임', '비주', '비중', '비즈', '빅머', '빅스텝망막', '사거리', '사람',
       '사이드카', '사자', '사자증가', '사창', '사흘', '산업', '산이', '삼성동', '삼성맨', '삼성전날',
       '삼성출발', '삼성코스닥', '상기에스', '상무부예측', '상서발생', '상속', '상승', '상승세', '상장',
       '상적', '상치', '상한매도', '샘피뉴트', '서동', '서비스연기금', '서희', '선도상장폐지', '선사로',
       '선임', '선출', '선택', '설명', '성가', '성장추가', '세로하락', '세미', '세상', '섹터지난해',
       '센트럴', '센티', '숏스', '수권', '수도', '수행', '순이', '슈빌', '스냅', '스랩', '스케',
       '스케개발', '스케기관', '스케바이', '스케상승', '스케외국인', '스케인사말', '스케종목', '스퀘어',
       '스타', '스텔', '승률가량', '시가총액러시아', '시가총액전날', '시간', '시기', '시대', '시도',
       '시마', '시브', '시비르', '신정은', '신축', '실적외국인', '심주', '쌍끌이', '쌍둥이', '아바스',
       '아빠', '아시아유가', '아이오', '아파트', '안이', '안팎상승', '안혜신', '안혜신미국', '안혜신세포',
       '알루', '알제', '압도수익률', '앞서', '약세외국인', '양은', '어가', '어제', '어치', '어치전자',
       '억만장자', '언스', '얼마', '업계', '업체', '에스', '에스티', '에이', '에이치', '에프',
       '엘리', '엘앤', '엠큐', '여기', '역대', '역시', '연간', '연결스케', '연구원', '연구원상승',
       '연구원제외', '연구원한국', '연기금명예', '연기금치료', '연매출', '연속', '연수구', '열사', '엿새',
       '영석', '영업', '영일만', '영향비비', '오딘', '오른', '오른쪽', '오은영', '오전', '오전관련',
       '오전급락', '오전박태진', '오전상반기', '오전서도', '오전외국인', '오전중국', '오전출발', '오후',
       '오후부회장', '오후전장', '오후코스닥', '오후회전', '오희', '오히려', '올해', '올해대부분',
       '외국인', '외국인넷마블', '외국인코스닥', '외국인콘퍼런스', '외인스케', '외인외국인', '우리', '우성',
       '우시', '움직종목', '월간', '웹툰대감', '위로미국', '위주', '위주외국인', '위주전명훈', '위해',
       '위협출발', '유가기념', '유가엘지', '유가징용', '유가측은', '유급', '유준', '유진희', '윤선',
       '윤정', '윤정분할', '윤필', '의무', '이광수', '이내', '이도연', '이드', '이명철', '이번',
       '이상', '이상미', '이상은', '이서현', '이슬기', '이엠', '이윤정', '이은정', '이은정매도',
       '이은정매수', '이정현', '이정현코스닥', '이제', '이지현', '이진욱', '이징', '이틀', '이후',
       '인구', '인조', '인천광역시', '인카', '인터콘티넨탈', '인플레', '인플릭시맙를', '인하진자', '일간',
       '일단', '일드', '일만', '일별', '일보', '일부', '일부반도체', '일삼', '임은진확대', '자기',
       '자산', '자청', '작년', '장주', '장현', '재는', '재차', '잭슨', '적용', '전날', '전날점유',
       '전날코로나', '전날한국', '전량폐경', '전명훈', '전일', '전일브렉', '전일우승', '전자신용',
       '전자중국', '전장', '전장밀러', '전장상승', '전장의약품', '전장통해', '전장허머', '전재욱', '전체',
       '전환', '접수', '정두리', '정오', '제과', '제시', '제약재개', '제약코로나', '제약하락', '제이',
       '조건', '조기', '조세백신', '조현영', '종가', '종가코바', '종로구검사', '종목', '종목공학',
       '종목법인세', '종목스케', '종목전체', '종목치료', '종목코로나', '종은', '종이', '종합매도',
       '종합무역', '종합자문', '종합종목', '종합처음', '종합코스닥', '종합회계', '주가', '주근', '주기도',
       '주로', '주말', '주식랠리', '주원', '주재', '주주', '주택', '줄줄이', '중등', '중심미국',
       '중이', '중인', '증권', '증시', '증시스케', '지금', '지난', '지난공사', '지난달', '지난생산',
       '지난외국인', '지난주', '지난집행', '지난해', '지난해투자', '지사', '지수', '지온', '지윤',
       '지윤하락', '지음', '지표유치', '직스', '직접', '진자', '진흥', '차잔', '채새롬', '처음',
       '청과', '청약', '청주시', '체라', '체육관', '초반', '초반무역', '초반민주당', '초반소액',
       '초반오후', '초반지점', '최근', '최근보합', '최대', '최정우백신', '최정희', '추락도시', '축소미국',
       '축소트러스트', '출발미국', '출발상승', '출발시마', '출발외국인', '출발임상시험', '출신', '출퇴근',
       '충북', '충북대학교', '측은', '치가', '침공배당', '칩스', '카불', '카운슬', '카카오나주',
       '카카오상승세', '카카오전체', '케다', '케이', '케이드', '케이엠', '코로나전날', '코비',
       '코스닥매수', '코스닥카카오', '코스피지수음성인식', '쿠폰', '퀸스', '크게', '타가', '타고', '타라',
       '탁금', '태현', '턱걸이순이익', '털썩', '텍메', '텍메드', '토대', '통해', '통화정책화학',
       '투연', '투자자기관', '투타', '트럼프트럼프', '티바', '티스', '파르', '파우치', '파이', '파적',
       '판사', '팔고', '팔자문제', '팔자미국', '페이', '포토', '포함', '폰데', '풀이', '프라',
       '프레', '프로비', '프리', '프티', '플라이', '피어', '피젠', '하나', '하락가상', '하락전장',
       '하락투자자', '하반기', '하우스', '학인', '학회', '한국', '한국인', '한라', '한상대', '한자리',
       '한편', '한해', '해당', '해도', '해리', '허쥬마', '헤럴드경제', '현재', '협상바이오', '협주',
       '혜미', '호실', '호조냉장', '호조발생', '호조주요', '호텔', '홀로', '홍콩달러', '화학채무',
       '화학합성', '확대정상회담', '환율유럽', '환율제약', '환자하락', '회복제약', '회복하락', '회사',
       '회하', '후시서비스', '훈풍전자', '훌쩍', '휘둘린', '흐름', '흥덕구']


stop_word = stop_word_public + company_stopword + stop_word_company 

In [ ]:
# 뉴스 날짜 dict

def news_keyword(keyword_pred, keyword_pred_date):
  news_date_dict = {}
  for news_idx in keyword_pred['news_id'].unique():
    df_temp = keyword_pred[keyword_pred['news_id'] == news_idx]
    temp_date = df_temp.iloc[0,0]
    news_date_dict[news_idx] = temp_date

  # 이진 분류 dict
  df_date_targetpm_dict = {}
  for idx in keyword_pred_date.index :
    df_date_targetpm_dict[keyword_pred_date.loc[idx,'date']] = int(keyword_pred_date.loc[idx, 'targetpm'])

  df_news = keyword_pred.groupby('news_id')[['sent']].sum()
  df_news_temp = keyword_pred.groupby('news_id')[['pred', 'date']].mean()
  df_news_temp['pred_pm'] = df_news_temp['pred'].apply(lambda x : 1 if x < 2 else 3)
  df_news['pred_pm'] = df_news_temp['pred_pm']
  df_news = df_news.reset_index()
  df_news['date'] = df_news['news_id'].apply(lambda x : news_date_dict[x])

  # 뉴스 기반으로 top20 먼저 추출

  news_id_list = df_news.index
  # 해당 뉴스 아이디 값산 토큰 
  for news in tqdm(news_id_list) :
    text_temp = df_sent[df_sent['news_id'] == news]['sent'].values
    news_len = len(df_sent[df_sent['news_id'] == news]['sent'].values)
    cmm_list = []

    # 문장단위 토큰
    for sent in text_temp :
      count_list = []
      text_noun = okt.nouns(sent)
      for nn in text_noun :
        if len(nn) >= 2 :
          count_list.append(nn)
      cnt1 = Counter(count_list)
      if len(cnt1) >= 20 :
        cnt1_cmm = cnt1.most_common(20)
      else :
        try : 
          cnt1_cmm = cnt1.most_common(10)
        except :
          try : 
            cnt1_cmm = cnt1.most_common(5)
          except :
            try : 
              cnt1_cmm = cnt1.most_common(3)
            except :
              cnt1_cmm = cnt1.most_common(1)

      cmm_list.append([tk[0] for tk in cnt1_cmm])
    total_list = []
    for news_token in cmm_list:
      for tok in news_token: 
        total_list.append(tok)
    cnt2 = Counter(total_list)
    cnt2_cmm = cnt2.most_common(20)
    top20_list = []
    top20 = cnt2_cmm[3:20]
    for tok in top20 :
      text, _ = tok
      if text not in stop_word  :
        top20_list.append(text)
    df_news.at[news,'top20_okt'] = ','.join(top20_list)

  # 날짜별 키워드 산출
  df_news_gr = df_news.groupby('date')[['top20_okt']].sum()
  df_news_gr_temp = df_news.groupby('date')[['top20_okt']].count()
  df_news_gr['news_cnt'] = df_news_gr_temp['top20_okt']
  for idx in tqdm(df_news_gr.index) :
    keyword_dict = {}
    for target in [1, 3]:
      df_temp = df_news[(df_news['date'] == idx) & (df_news['pred_pm'] == target)]
      news_cnt = len(df_news[(df_news['date'] == idx) & (df_news['pred_pm'] == target)])
      df_temp_gr = df_temp.groupby('date')[['top20_okt']].sum()
      if target == 1:
        target_col = '부정'
      else :
        target_col = '긍정'
      if len(df_temp_gr) != 0:
        cnt_temp = Counter(df_temp_gr['top20_okt'].values[0].split(','))
        cnt_temp_cmm = cnt_temp.most_common(100)
        text_list = []
        cnt_list = []
        for text, cnt in cnt_temp_cmm[:10] :
          if text not in stop_word  :
            text_list.append(text) 
          cnt_list.append(str(round(cnt/news_cnt, 2)))
        if text_list != []:
          keyword_dict[f"{target_col}_단어"] = ",".join(text_list)
          # keyword_dict[f"{target_col}_빈도"] = ",".join(cnt_list)
        else :
          keyword_dict[f"{target_col}_단어"] = ""
          # keyword_dict[f"{target_col}_빈도"] = ""
      else :
        keyword_dict[f"{target_col}_단어"] = ""
        # keyword_dict[f"{target_col}_빈도"] = ""
    df_news_gr.loc[idx, 'keyword'] = '한글'
    df_news_gr.at[idx, 'keyword'] = keyword_dict
  df_news_gr = df_news_gr.reset_index()
  df_news_gr['target_pm'] =  df_news_gr['date'].apply(lambda x : df_date_targetpm_dict[x])
  return df_news_gr


In [ ]:
save_path = f'/content/drive/MyDrive/뉴스원인분석_modeling/final_news_modeling/{company_name}/{company_name}'
save_df_sent_path = f'/content/drive/MyDrive/뉴스원인분석_modeling/final_news_modeling/{company_name}/{company_name}_sent_split.csv'
keyword_pred = pd.read_csv(save_path + f'_test_pred_by_sent.csv')
keyword_pred_date = pd.read_csv(save_path + f'_test_pred_by_date_total.csv')
df_sent = pd.read_csv(save_df_sent_path)

In [ ]:
df_keyword = news_keyword(keyword_pred, keyword_pred_date)
df_keyword

  0%|          | 0/5718 [00:00<?, ?it/s]

  0%|          | 0/206 [00:00<?, ?it/s]

,date,top20_okt,news_cnt,keyword,target_pm
0,2022-01-28,"시가총액,제외,연초,자금,블랙홀차관,토론,양자,민주당,진행,입장,국민,대선,재판,검...",72,"{'부정_단어': '하락,시가총액,삼성,사업,투자,미국,전망,실적,자금,분할', '...",1
1,2022-02-03,"정부,생산,공급,자율,소재,재활용,플라스틱,전망,제휴,네이버,주머니,유럽,리스크첫날...",112,"{'부정_단어': '', '긍정_단어': '하락,미국,투자,배터리,긴축,금리,생산,...",3
2,2022-02-04,"배터리,스테인리스강,세계,수요,전기차,원료,경기,미국,비량,차지대형,뱅크,증가,사상...",39,"{'부정_단어': '하락,매매,연휴', '긍정_단어': '배터리,미국,전기차,계약,...",3
3,2022-02-07,"하락,출발,중심,전자,삼성,대형주계약,배터리,리튬,첫날,편입,평가,생산,공급망,과정...",61,"{'부정_단어': '', '긍정_단어': '미국,투자,삼성,배터리,전자,자금,글로벌...",3
4,2022-02-08,"전망,투자,상황,하락,긴축,미국,바닥,세계,정책,심리,보통금통위,금리,스프레드,투자...",68,"{'부정_단어': '투자,증가,확대,글로벌,전기차,이익,자금,목표,배터리,미국', ...",1
...,...,...,...,...,...
201,2022-11-24,"미국,환율민주당,투자,결과,혜주,법안,장기,관점,상원,승리,공화당,신재생에너지,기대...",35,"{'부정_단어': '', '긍정_단어': '주식,투자,전자,차지,구매,보합,삼성,상...",3
202,2022-11-25,"출발,물가,발표,압력,뉴욕,나스닥,대유행고가,경신,특징,신고돌입,합작,얼티엄셀즈,공...",34,"{'부정_단어': '생산,배터리,공급,확보,파스,미네랄,미국,계약,체결,탄산', '...",3
203,2022-11-28,"사업,도청,체결,건설,기술,행정,서비스,발리,협력,김영섭,보르네오섬,글로벌하락,보합...",28,"{'부정_단어': '전자,삼성,재계,재무,생산,전기차,참여,투자,주식,확대', '긍...",1
204,2022-11-29,"현금,추진,사업,당시,자금,방식,상황,건설,보유,수익이재성,효율,기술,개발,본부장,...",15,"{'부정_단어': '', '긍정_단어': '사업,전자,부문,기술,수상,미국,배터리,...",3


In [ ]:
for idx in df_keyword.sample(frac=1).index[:50] :
  temp_neg = df_keyword.loc[idx, 'keyword']['부정_단어']
  temp_pos = df_keyword.loc[idx, 'keyword']['긍정_단어']
  print(idx, df_keyword.loc[idx, 'date'])
  print('부정단어 :',temp_neg, '\n', '긍정_단어 :', temp_pos, '\n\n')

7 2022-02-11
부정단어 :  
 긍정_단어 : 금융,지주,투자,하락,특징,메리츠화재,정기,더존,제약,미국 


111 2022-07-14
부정단어 : 삼성,하락,거래소,소재,특징,이익,매출,중국,증가,배터리 
 긍정_단어 : 매출,이익,하락,실적,전자,화학,보호,예수,해제,목표 


45 2022-04-08
부정단어 : 인재,영상,유튜브,신입사원,절차,직원,확보,코믹,인사,제작 
 긍정_단어 : 하락,출발,미국,투자,보합,삼성,전기차,차량,운용,특징 


151 2022-09-13
부정단어 :  
 긍정_단어 : 미국,생산,배터리,전기차,법인,현대차,투자,환율,인상,공장 


175 2022-10-19
부정단어 :  
 긍정_단어 : 동기,실적,연결,매출,증가,지원,전기차,첫날,바이오,공시 


131 2022-08-11
부정단어 : 상반기,수주,디스플레이,설립,증가,생산,해외,글로벌,시장점유율,세계 
 긍정_단어 : 상반기,배터리,생산,미국,중국,사업,성장,글로벌,신규,현대차 


30 2022-03-18
부정단어 :  
 긍정_단어 : 전기차,판매,현지,자동차,최초,준공,정부,전략,완성,공장 


97 2022-06-24
부정단어 : 투자,확대,유치,공장,미국,세계,대통령,허브,자카르타,자동차 
 긍정_단어 : 전자,코스닥,출발,출하,가격,대표,경영인,경영,전당,연방 


10 2022-02-16
부정단어 : 가격,미국,글로벌,원자재,탑재,출시,전자,세계,광물,안정 
 긍정_단어 : 전기차,하락,글로벌,탑재,이익,유가,지정학적,세계,카카오,전기 


161 2022-09-27
부정단어 : 투자,미국,삼성,이익,매출,사업,전기차,포럼,코트라,진출 
 긍정_단어 :  


120 2022-07-27
부정단어 : 하락,강세,우위,소프트웨어,미국,지원스탠퍼드대,생산,협력,아시아,지속가능성 
 긍정_단어 : 공급,투자,전자,마하,증설,라인,미국,전기차,판매량,배터리 


23 2022-03-08
부정단어 : 생산,우크라이나,하락,목표,의장,배터리,글로벌,공

In [ ]:
save_path = f'/content/drive/MyDrive/뉴스원인분석_modeling/final_news_modeling/{company_name}/{company_name}'
import os
os.makedirs(save_path, exist_ok=True)
df_keyword.to_csv(save_path + '_news_keyword.csv', index=False)

# 댓글 분석


In [ ]:
!pip install transformers
!pip install attrdict 
!pip install konlpy
!pip install 'git+https://github.com/krevas/huggingface_konlpy'
!pip install ruamel-yaml
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/krevas/huggingface_konlpy to /tmp/pip-req-build-oaupi15u
  Running command git clone --filter=blob:none --quiet https://github.com/krevas/huggingface_konlpy /tmp/pip-req-build-oaupi15u
  Resolved https://github.com/krevas/huggingface_konlpy to commit 00af654a4a24563b4df0a8903faef2121dbc8d90
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 555.3/555.3 KB 16.

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
import warnings
warnings.filterwarnings(action='default')

In [ ]:
def preprocessing_train_save(df_bc):
  df_bc = df_bc[['date_cate','content', 'reaction']]
  df_bc = df_bc.rename(columns={'date_cate' : 'date', 'reaction' : 'target'})
  df_bc['recontent'] = df_bc['content'].apply(lambda x : re.sub('[^a-zA-Z가-힣ㄱ-ㅎ.0-9 ]', '', x))
  df_bc['recontent'] =df_bc['recontent'].apply(lambda x : re.sub(' +', ' ', x))
  df_bc = df_bc[(df_bc['recontent'] != '') & (df_bc['recontent'] != ' ') & (df_bc['recontent'] != '  ')]
  df_bc = df_bc.sample(frac=1, random_state=2022)

  df_gr = df_bc.groupby('date')[['target']].mean()
  # 날짜별로 train_test_split
  train_date, test_date = train_test_split(df_gr, test_size=0.1, random_state=2022, stratify=df_gr['target'])
  train_date_list = train_date.index
  test_date_list = test_date.index
  # 라벨링 문자로
  df_dict = {'0' : '하락', '1' : '상승'}
  train = df_bc[df_bc['date'].isin(train_date_list)][['date','recontent', 'target']]
  test = df_bc[df_bc['date'].isin(test_date_list)][['date','recontent', 'target']]
  train['target'] = train['target'].apply(lambda x : df_dict[str(x)])
  test['target'] = test['target'].apply(lambda x : df_dict[str(x)])
  train_raw = train.copy()
  test_pred = test.copy()
  train = train[['recontent', 'target', 'date']]
  test = test[['recontent', 'target', 'date']]

  train_sampling_up = train[train['target'] == '상승'].sample(10000, random_state=1)
  train_sampling_down = train[train['target'] == '하락'].sample(10000, random_state=1)
  train_sampling = pd.concat([train_sampling_up, train_sampling_down])[['recontent', 'target']]
  train_not_sampled = train.loc[~train.index.isin(train_sampling.index), :]
  train_sampling = train_sampling.sample(frac=1 , random_state=2022)
  return train_sampling , train_not_sampled, test[['recontent', 'target']], test

def board_pred(model, test):
  model.eval()
  from transformers import TextClassificationPipeline
  sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model)

  for idx in tqdm(test.index) :
    review_list = [test.loc[idx,'recontent']]
    for review in review_list:
      pred = sentiment_classifier(review[:128])
      pred_label = pred[0]['label'][-1]
      pred_score = pred[0]['score']
      test.loc[idx,'pred_label'] = pred_label
      test.loc[idx,'score'] = pred_score
  return test

In [ ]:
df_bc

,code,title,content,date,like,unlike,view_count,date_cate,전일비_비율,reaction
0,373220,나쁘지 않은 정보,여기서 공유하고 있습니다.소통도 원활하게 진행중입니다.함께해요.kakaoshorts...,2022-11-30 16:03:00,2,1,225,2022-11-30,3.53,1
1,373220,외인입니다. 입장 밝힙니다.,VI.......ctory,2022-11-30 15:52:00,2,2,508,2022-11-30,3.53,1
2,373220,370조 수주잔고 ~,앞으로 고고싱싱 !,2022-11-30 16:03:00,6,1,277,2022-11-30,3.53,1
3,373220,33% 공매도 거래! 외인숏커버 물량,이건 시간외 또는 내일 또 오른다는 ~~,2022-11-30 15:56:00,6,1,473,2022-11-30,3.53,1
4,373220,내일도,이어 달리기 하겠지,2022-11-30 16:04:00,7,0,237,2022-11-30,3.53,1
...,...,...,...,...,...,...,...,...,...,...
63641,373220,소문난 잔치,주식하루이틀허냐 빨간봉 한두개보임 팔어라물량투하 눈에 보이는구만 17만에 입성하마,2022-01-27 08:23:00,2,2,78,2022-01-27,0.00,1
63642,373220,적정주가 63만원,적정주가 63만원,2022-01-27 08:23:00,1,2,93,2022-01-27,0.00,1
63643,373220,오전에 매수세 겁나게 들어올거다,가지고만 있는놈이 승자다,2022-01-27 08:24:00,1,2,75,2022-01-27,0.00,1
63644,373220,따상상상 갑니다,내일 100만원에 팔면 됩니다 ㄱㄱㄱ,2022-01-27 08:24:00,3,1,290,2022-01-27,0.00,1


In [ ]:
# 회사명에 따른 경로 설정 변경
%cd /content/drive/MyDrive/뉴스원인분석_modeling/final_news_modeling/lgenergy/Dialog-KoELECTRA_light/dialog_electra/finetune/dialog_electra/finetune/data/hate-speech

train, train_not_sampled, test, test_pred = preprocessing_train_save(df_bc)
train.to_csv('train.tsv', sep='\t', index=False)
test.to_csv('val.tsv', sep='\t', index=False)


%cd /content/drive/MyDrive/뉴스원인분석_modeling/final_news_modeling/lgenergy/Dialog-KoELECTRA_light/dialog_electra/finetune
!python3 run_finetune.py --config_file conf/hate-speech/electra-small.yaml

/content/drive/MyDrive/뉴스원인분석_modeling/final_news_modeling/lgenergy/Dialog-KoELECTRA_light/dialog_electra/finetune/dialog_electra/finetune/data/hate-speech
/content/drive/MyDrive/뉴스원인분석_modeling/final_news_modeling/lgenergy/Dialog-KoELECTRA_light/dialog_electra/finetune
2023-01-15 11:53:58.438592: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/usr/local/lib/python3.8/dist-packages/ruamel/yaml/constructor.py:532: MantissaNoDotYAML1_1Warning: 
In YAML 1.1 floating point values should have a dot ('.') in their mantissa.
See the Floating-Point Language-Independent Type for YAML™ Version 1.1 specification
( http://yaml.org/type/float.html ). This dot is not required for JSON nor for YAML 1.2

Correct your float: "1e-8" on line: 18, column: 14

or alternat

In [ ]:
# 마지막 체크포인트 모델 확인 후 저장
import torch
from transformers import ElectraTokenizer, ElectraForSequenceClassification
model = ElectraForSequenceClassification.from_pretrained("skplanet/dialog-koelectra-small-discriminator", num_labels=2)
tokenizer = ElectraTokenizer.from_pretrained("skplanet/dialog-koelectra-small-discriminator")
model.load_state_dict(torch.load('./dialog_electra/finetune/model/ep30sample/hate-speech_small_v1/checkpoint-800/pytorch_model.bin'))

torch.save(model, f'/content/drive/MyDrive/뉴스원인분석_modeling/final_news_modeling/{company_name}/{company_name}_board_comment_ep20.pt')
torch.save(model.state_dict(),f'/content/drive/MyDrive/뉴스원인분석_modeling/final_news_modeling/{company_name}/{company_name}_board_comment_ep20_state_dict.bin')

Some weights of the model checkpoint at skplanet/dialog-koelectra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at skplanet/dialog-koelectra-small-discriminator and are newly ini

In [ ]:
# 전체 결과값 예측
df_total = pd.concat([train_not_sampled, test_pred])
board_total_pred = board_pred(model, df_total)
board_total_pred

  0%|          | 0/43621 [00:00<?, ?it/s]

,recontent,target,date,pred_label,score
48915,한국에있다 여기는중국인가,하락,2022-01-28,0,0.848676
27412,어여 나오길.,하락,2022-03-07,1,0.988648
4805,러시아군 지속적인 후퇴는 핵무기 사용을 위한 것일어나믄 안되는 일이 일어날라고 한다...,상승,2022-10-05,0,0.988855
26694,3000이라도 올리라고혀봐.ㅊㅊㅊ.삭열이틀렸어..,하락,2022-03-11,1,0.991375
60016,지금 나스닥도 박고 있고 코스피도 박고 있는데이렇게 시총 큰 애가 갈 거라고 생각하냐,상승,2022-01-27,1,0.665379
...,...,...,...,...,...
13392,아직도 시총 90조라니....lg화학 반성해라 자기들끼리 헤쳐먹고 주주들 배신때리고...,하락,2022-06-30,0,0.980204
41063,기원하네,상승,2022-02-07,1,0.900274
43777,msci편입으로 55006900억이 수급 들어온다는데이게 보통 장마감하면 수급이 개...,상승,2022-02-04,0,0.989691
43035,60층본다,상승,2022-02-04,0,0.672662


In [ ]:
board_total_pred['pred_label'] = board_total_pred['pred_label'].astype('int')
board_total_pred_gr = board_total_pred.sort_values('date').groupby('date')[['pred_label']].mean()
board_total_pred_gr = board_total_pred_gr.reset_index()
for idx in board_total_pred_gr.index :
  date = board_total_pred_gr.loc[idx, 'date']
  neg_cnt = len(board_total_pred[(board_total_pred['date'] == date) & (board_total_pred['pred_label'] == 0)])
  pos_cnt = len(board_total_pred[(board_total_pred['date'] == date) & (board_total_pred['pred_label'] == 1)])
  sum_cnt = neg_cnt + pos_cnt
  board_total_pred_gr.loc[idx, 'neg_cnt'] = neg_cnt / sum_cnt
  board_total_pred_gr.loc[idx, 'pos_cnt'] = pos_cnt / sum_cnt
board_total_pred_gr['year'] = board_total_pred_gr['date'].apply(lambda x : pd.to_datetime(x).isocalendar()[0])
board_total_pred_gr['week'] = board_total_pred_gr['date'].apply(lambda x : pd.to_datetime(x).isocalendar()[1])
board_total_pred_gr['day'] = board_total_pred_gr['date'].apply(lambda x : pd.to_datetime(x).isocalendar()[2])
total_day_week = board_total_pred_gr.groupby(['year', 'week'])[['neg_cnt', 'pos_cnt']].mean()
for idx in board_total_pred_gr.index :
  year = board_total_pred_gr.loc[idx,'year']
  week = board_total_pred_gr.loc[idx,'week']
  try :
    board_total_pred_gr.loc[idx, 'last_week'] = '-1'
    last_dict = {}
    last_dict['neg'] = round(total_day_week.loc[(year,week -1), 'neg_cnt'],2)
    last_dict['pos'] = round(total_day_week.loc[(year,week -1), 'pos_cnt'],2)
    board_total_pred_gr.at[idx, 'last_week'] = last_dict
  except :
    board_total_pred_gr.loc[idx, 'last_week'] = '-1'
board_total_pred_gr

,date,pred_label,neg_cnt,pos_cnt,year,week,day,last_week
0,2022-01-27,0.586125,0.413875,0.586125,2022,4,4,-1
1,2022-01-28,0.464843,0.535157,0.464843,2022,4,5,-1
2,2022-02-03,0.558496,0.441504,0.558496,2022,5,4,"{'neg': 0.47, 'pos': 0.53}"
3,2022-02-04,0.577187,0.422813,0.577187,2022,5,5,"{'neg': 0.47, 'pos': 0.53}"
4,2022-02-07,0.591686,0.408314,0.591686,2022,6,1,"{'neg': 0.43, 'pos': 0.57}"
...,...,...,...,...,...,...,...,...
202,2022-11-24,0.454545,0.545455,0.454545,2022,47,4,"{'neg': 0.45, 'pos': 0.55}"
203,2022-11-25,0.476190,0.523810,0.476190,2022,47,5,"{'neg': 0.45, 'pos': 0.55}"
204,2022-11-28,0.557377,0.442623,0.557377,2022,48,1,"{'neg': 0.51, 'pos': 0.49}"
205,2022-11-29,0.500000,0.500000,0.500000,2022,48,2,"{'neg': 0.51, 'pos': 0.49}"


In [ ]:
# 댓글 전체 예측값 저장
board_total_pred_gr.to_csv(f'/content/drive/MyDrive/뉴스원인분석_modeling/final_news_modeling/{company_name}/{company_name}'  + '_board_pred.csv', index=False)

# 전체 데이터 저장

In [ ]:
save_path = f'/content/drive/MyDrive/뉴스원인분석_modeling/final_news_modeling/{company_name}/{company_name}'
board_total_pred_gr = pd.read_csv(save_path  + '_board_pred.csv')
df_keyword = pd.read_csv(save_path + '_news_keyword.csv')
keyword_pred_date = pd.read_csv(save_path + f'_test_pred_by_date_total.csv')
import datetime
from tqdm.notebook import tqdm
bd_m1 = board_total_pred_gr[board_total_pred_gr['last_week'] == '-1']

# 전주차 댓글 없는 경우 디버깅
for idx in tqdm(bd_m1.index) :
  temp_date = bd_m1.loc[idx,'date']
  last_week = (pd.to_datetime(temp_date) - datetime.timedelta(weeks=1)).strftime('%Y-%m-%d')
  last_calender = pd.to_datetime(last_week).isocalendar()
  last_year = last_calender[0]
  last_week = last_calender[1]
  df_temp = board_total_pred_gr[(board_total_pred_gr['year'] == last_year) & (board_total_pred_gr['week'] == last_week)]
  if len(df_temp) == 0 or str(df_temp['neg_cnt']) == 'nan':
    board_total_pred_gr.loc[idx, 'last_week'] = '-1'
  else :
    last_neg = df_temp['neg_cnt'].mean()
    last_pos = df_temp['pos_cnt'].mean()
    temp_dict = {}
    temp_dict['neg'] = round(last_neg, 2)
    temp_dict['pos'] = round(last_pos, 2)
    board_total_pred_gr.at[idx, 'last_week'] = temp_dict

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# 머지
merge_total = pd.merge(keyword_pred_date, board_total_pred_gr, how='left', on='date')
merge_total = pd.merge(merge_total, df_keyword, how='left', on='date')
merge_total.to_csv(save_path + '_data_total.csv', index=False)
merge_total

,date,target,news_pred,targetpm,cnt_pred,pred_target,pred_pm,eval_5,3target,pred_3target,...,neg_cnt,pos_cnt,year,week,day,last_week,top20_okt,news_cnt,keyword,target_pm
0,2022-01-28,0.0,0.000000,1.0,0.0,0,1,True,1.0,1,...,0.535157,0.464843,2022,4,5,-1,"시가총액,제외,연초,자금,블랙홀차관,토론,양자,민주당,진행,입장,국민,대선,재판,검...",72,"{'부정_단어': '하락,시가총액,삼성,사업,투자,미국,전망,실적,자금,분할', '...",1
1,2022-02-03,4.0,3.997347,3.0,4.0,4,3,True,3.0,3,...,0.441504,0.558496,2022,5,4,"{'neg': 0.47, 'pos': 0.53}","정부,생산,공급,자율,소재,재활용,플라스틱,전망,제휴,네이버,주머니,유럽,리스크첫날...",112,"{'부정_단어': '', '긍정_단어': '하락,미국,투자,배터리,긴축,금리,생산,...",3
2,2022-02-04,4.0,3.939130,3.0,4.0,4,3,True,3.0,3,...,0.422813,0.577187,2022,5,5,"{'neg': 0.47, 'pos': 0.53}","배터리,스테인리스강,세계,수요,전기차,원료,경기,미국,비량,차지대형,뱅크,증가,사상...",39,"{'부정_단어': '하락,매매,연휴', '긍정_단어': '배터리,미국,전기차,계약,...",3
3,2022-02-07,4.0,4.000000,3.0,4.0,4,3,True,3.0,3,...,0.408314,0.591686,2022,6,1,"{'neg': 0.43, 'pos': 0.57}","하락,출발,중심,전자,삼성,대형주계약,배터리,리튬,첫날,편입,평가,생산,공급망,과정...",61,"{'부정_단어': '', '긍정_단어': '미국,투자,삼성,배터리,전자,자금,글로벌...",3
4,2022-02-08,1.0,0.985011,1.0,1.0,1,1,True,1.0,1,...,0.503724,0.496276,2022,6,2,"{'neg': 0.43, 'pos': 0.57}","전망,투자,상황,하락,긴축,미국,바닥,세계,정책,심리,보통금통위,금리,스프레드,투자...",68,"{'부정_단어': '투자,증가,확대,글로벌,전기차,이익,자금,목표,배터리,미국', ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,2022-11-24,3.0,3.000000,3.0,3.0,3,3,True,3.0,3,...,0.545455,0.454545,2022,47,4,"{'neg': 0.45, 'pos': 0.55}","미국,환율민주당,투자,결과,혜주,법안,장기,관점,상원,승리,공화당,신재생에너지,기대...",35,"{'부정_단어': '', '긍정_단어': '주식,투자,전자,차지,구매,보합,삼성,상...",3
202,2022-11-25,2.0,2.055118,3.0,2.0,2,3,True,2.0,2,...,0.523810,0.476190,2022,47,5,"{'neg': 0.45, 'pos': 0.55}","출발,물가,발표,압력,뉴욕,나스닥,대유행고가,경신,특징,신고돌입,합작,얼티엄셀즈,공...",34,"{'부정_단어': '생산,배터리,공급,확보,파스,미네랄,미국,계약,체결,탄산', '...",3
203,2022-11-28,1.0,1.000000,1.0,1.0,1,1,True,1.0,1,...,0.442623,0.557377,2022,48,1,"{'neg': 0.51, 'pos': 0.49}","사업,도청,체결,건설,기술,행정,서비스,발리,협력,김영섭,보르네오섬,글로벌하락,보합...",28,"{'부정_단어': '전자,삼성,재계,재무,생산,전기차,참여,투자,주식,확대', '긍...",1
204,2022-11-29,3.0,3.000000,3.0,3.0,3,3,True,3.0,3,...,0.500000,0.500000,2022,48,2,"{'neg': 0.51, 'pos': 0.49}","현금,추진,사업,당시,자금,방식,상황,건설,보유,수익이재성,효율,기술,개발,본부장,...",15,"{'부정_단어': '', '긍정_단어': '사업,전자,부문,기술,수상,미국,배터리,...",3


In [ ]:
# DB와 형식 맞추기
from datetime import datetime 
merge_total['date'] = pd.to_datetime(merge_total['date'])
merge_total['code'] = company_code
merge_total['updated_date'] = datetime.today().strftime('%Y-%m-%d')
merge_total.rename(columns={'keyword' : 'news_keyword', 'last_week': 'comment'}, inplace=True)
merge_total_db = merge_total[['date','code','news_keyword','comment', 'updated_date']]
merge_total_db.to_csv(save_path + f'_total_data_for_db.csv', index=False)
merge_total_db

,date,code,news_keyword,comment,updated_date
0,2022-01-28,373220,"{'부정_단어': '하락,시가총액,삼성,사업,투자,미국,전망,실적,자금,분할', '...",-1,2023-01-18
1,2022-02-03,373220,"{'부정_단어': '', '긍정_단어': '하락,미국,투자,배터리,긴축,금리,생산,...","{'neg': 0.47, 'pos': 0.53}",2023-01-18
2,2022-02-04,373220,"{'부정_단어': '하락,매매,연휴', '긍정_단어': '배터리,미국,전기차,계약,...","{'neg': 0.47, 'pos': 0.53}",2023-01-18
3,2022-02-07,373220,"{'부정_단어': '', '긍정_단어': '미국,투자,삼성,배터리,전자,자금,글로벌...","{'neg': 0.43, 'pos': 0.57}",2023-01-18
4,2022-02-08,373220,"{'부정_단어': '투자,증가,확대,글로벌,전기차,이익,자금,목표,배터리,미국', ...","{'neg': 0.43, 'pos': 0.57}",2023-01-18
...,...,...,...,...,...
201,2022-11-24,373220,"{'부정_단어': '', '긍정_단어': '주식,투자,전자,차지,구매,보합,삼성,상...","{'neg': 0.45, 'pos': 0.55}",2023-01-18
202,2022-11-25,373220,"{'부정_단어': '생산,배터리,공급,확보,파스,미네랄,미국,계약,체결,탄산', '...","{'neg': 0.45, 'pos': 0.55}",2023-01-18
203,2022-11-28,373220,"{'부정_단어': '전자,삼성,재계,재무,생산,전기차,참여,투자,주식,확대', '긍...","{'neg': 0.51, 'pos': 0.49}",2023-01-18
204,2022-11-29,373220,"{'부정_단어': '', '긍정_단어': '사업,전자,부문,기술,수상,미국,배터리,...","{'neg': 0.51, 'pos': 0.49}",2023-01-18


In [ ]:
merge_total_db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206 entries, 0 to 205
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          206 non-null    datetime64[ns]
 1   code          206 non-null    object        
 2   news_keyword  206 non-null    object        
 3   comment       206 non-null    object        
 4   updated_date  206 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 9.7+ KB


In [ ]:
merge_total_db[merge_total_db['comment'] == '-1']

,date,code,news_keyword,comment,updated_date
0,2022-01-28,373220,"{'부정_단어': '하락,시가총액,삼성,사업,투자,미국,전망,실적,자금,분할', '...",-1,2023-01-18


In [ ]:
merge_total_db.loc[merge_total_db['comment'].isna(),:]

,date,code,news_keyword,comment,updated_date


In [ ]:
# 태블로 데이터 저장
merge_total['tableau_pred'] = (merge_total['pred_target'] - 2) / 2
merge_total_tableau = merge_total[['date', 'tableau_pred']]
merge_total_tableau['news_cnt'] = df_sent.sort_values('date').groupby('date')[['news_id']].nunique()['news_id'].values
merge_total_tableau.to_csv(save_path + f'_test_pred_for_tableau.csv', index=False)
merge_total_tableau

<ipython-input-33-bfb0385aabd8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_total_tableau['news_cnt'] = df_sent.sort_values('date').groupby('date')[['news_id']].nunique()['news_id'].values


,date,tableau_pred,news_cnt
0,2022-01-28,-1.0,79
1,2022-02-03,1.0,122
2,2022-02-04,1.0,40
3,2022-02-07,1.0,69
4,2022-02-08,-0.5,68
...,...,...,...
201,2022-11-24,0.5,36
202,2022-11-25,0.0,38
203,2022-11-28,-0.5,29
204,2022-11-29,0.5,16
